# Requirements

# Imports

In [106]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import configparser
import mysql.connector
import json

#Hadoop
from hdfs import InsecureClient


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 48288)
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/usr/local/anaconda3/lib/python3.9/site-packages/pyspark/accum

In [2]:
import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 

# Functions

## General

In [3]:
def check_folder(name_folder):            
    #Creating folder if that doesn't exist
    p = %pwd
    p = p + f'/{name_folder}'
    path = os.path.expanduser(p)
    if not os.path.exists(path):
        os.makedirs(path)
        print("{} created.".format(path))
        
    return path

In [4]:
def get_file_name(item_name, ext='*tar'):
    
    file_names = [f.name for f in internetarchive.get_files(item_name, glob_pattern= ext)]
    
    return file_names

In [5]:
def humanize(size_bytes):
    KB = 1 << 10
    MB = 1 << 20
    GB = 1 << 30

    if size_bytes < KB:
        return '{} B'.format(size_bytes)
    elif size_bytes < MB:
        return '{:.2f} KiB'.format(size_bytes/KB)
    elif size_bytes < GB:
        return '{:.2f} MiB'.format(size_bytes/MB)
    else:
        return '{:.2f} GiB'.format(size_bytes/GB)


## Cleaning and Transfomations

In [6]:
def get_data(file_path, file_name, word_bag = False, lang = False, db_update = False):
    
    #reading json
    df = pd.read_json(file_path, lines=True, compression='gzip')
    
    #updating total tweets
    if not update_db == False:
        update_db(table = 'control', field = f"count_total = '{len(df)}'", condition = f"name = '{file_name}'")
    
    #Filtering language if a language was sent
    if not lang == False:
        df = df[df.lang == lang]
    
    #taking away columns unnecessary
    df = df.loc[:, ['created_at', 'text', 'entities']]
    
    #filtering Tweets using subject word bag if a word bag was sent
    if not word_bag == False:
        bow = '|'.join(word_bag) # bag of word
        df = df[df['text'].str.contains(bow, case=False)]
        
        #updating filtered tweets
        if not update_db == False:
            update_db(table = 'control', field = f"count_filtered = '{len(df)}'", condition = f"name = '{file_name}'")
    
    return df

## Control Operations

### CSV

In [7]:
def control_op_csv(control_file):

    #if file doesn't exist I'll create it
    if not os.path.exists(control_file):
        with open(control_file, 'w', newline='') as control_csv:
            writer = csv.DictWriter(control_csv, fieldnames=['name','datetime'])
            writer.writeheader()
        control_csv.close()
        print("{} created.".format(control_file))
            
    #Reading the control file
    with open(control_file, 'r', newline='') as control_csv:
        reader = csv.DictReader(control_csv)
        reader_data = [r for r in reader]
        control_csv.seek(0)
    control_csv.close()
    
    return reader_data

In [8]:
def download_file_csv(item_name, file_name, download_folder = 'Downloads'):
    
    path = check_folder(download_folder)
    
    #calling function for getting control file
    control_path = path + '/control_download.csv'    
    reader_data = control_op_csv(control_path)
    
    #Getting Itens names
    item = internetarchive.get_item(item_name)
        
    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_download:
        writer = csv.DictWriter(control_download, fieldnames=['name','datetime'])
        
        #check if the file has been downloaded before
        if not any(row['name'] == file_name for row in reader_data):
            
            #downloading
            r = item.download(
                destdir=path,  # The directory to download files to
                ignore_existing=True,  # Skip files that already exist locally
                checksum=True,  # Skip files based on checksum
                verbose=True,  # Print progress to stdout
                retries=100,  # The number of times to retry on failed requests
                no_directory=True,  # Download withtout the identifier
                files = file_name)
            
            #Adding file name on control list
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
    control_download.close()
    
    path_file = path + f'/{file_name}'
    
    return path_file

In [9]:
def tar_file_csv(tar_file, file_name, extration_folder='Extraction'):

    path = check_folder(extration_folder)
    
    #calling function for getting control file
    control_path = path + '/control_extraction.csv'    
    reader_data = control_op(control_path)

    #openning control file to add file names on the list
    with open(control_path, 'a', newline='') as control_tar:
        writer = csv.DictWriter(control_tar, fieldnames=['name','datetime'])
        
        #check if the file has been extrated before
        if not any(row['name'] == file_name for row in reader_data):

            #Extracting file
            tar_file.extract(file_name, path=path)

            #Saving files name on control file
            row = {'name' : file_name, 'datetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
            writer.writerow(row)
            
            path_file = path + f'/{file_name}'
            
        else:
            
            path_file = False
    
    control_tar.close()
    
    
    return path_file

### Mysql

#### Basic operations

In [10]:
def open_connection():

    #getting configuration
    config = configparser.ConfigParser()
    config.read('config.ini')

    # Connect to Mysql
    conn = mysql.connector.connect(
        host = config['mysql']['host'],
        user = config['mysql']['user'],
        password = config['mysql']['password'],
        database = config['mysql']['database']
    )
        
    return conn 

In [11]:
def update_db(table, field, condition):
    
    conn = open_connection()
    cursor = conn.cursor()

    cursor.execute(f"UPDATE {table} SET {field} WHERE {condition};")

    conn.commit()
    cursor.close()
    conn.close()

In [12]:
def query_db(query):
    
    conn = open_connection()
    cursor = conn.cursor()
    
    cursor.execute(query)

    conn.commit()
    cursor.close()
    conn.close()

In [13]:
def read_db(query):
    conn = open_connection()
    
    df = pd.read_sql_query(query, conn)
    
    conn.close()
    return df

#### Specific operations

In [104]:
def check_control_db(file_type, file_name = False):
    
    if file_name == False:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}'").to_dict(orient='records')
    
    else:
        
        check = read_db(f"SELECT name FROM control WHERE type ='{file_type}' AND name = '{file_name}'"
                       ).to_dict(orient='records')
        
        if len(check) > 0:
            check = True
        else:
            check = False

    return check

In [105]:
def download_file_db(item_name, file_name, download_folder = 'Downloads', type_control = 'download'):
    
    path = check_folder(download_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
                             
    #check if the file has been unzipped before
    if check == False:
    
        #getting Itens names
        item = internetarchive.get_item(item_name)

        #downloading
        r = item.download(
            destdir=path,  # The directory to download files to
            ignore_existing=True,  # Skip files that already exist locally
            checksum=True,  # Skip files based on checksum
            verbose=False,  # Print progress to stdout
            retries=100,  # The number of times to retry on failed requests
            no_directory=True,  # Download withtout the identifier
            files = file_name)
        
        #getting Metadata
        metadata = list(filter(lambda p: p["name"] == file_name, item.item_metadata['files']))

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = metadata[0]['size']
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}','{dt}','{type_control}','{sz}')")
    
        path_file = path + f'/{file_name}'
        
    else:
        
        path_file = False   
    
    return path_file

In [16]:
def tar_file_db(tar_file, file_name, extration_folder='Extraction', type_control = 'extraction'):
    
    path = check_folder(extration_folder)
    
    #query to get information from Mysql about control
    check = check_control_db(file_type = type_control, file_name = file_name)
    
    
    #check if the file has been unzipped before
    if check == False:

        #Extracting file
        tar_file.extract(file_name, path=path)

        #Adding file name on control list
        dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        sz = tar_file.getmember(name=file_name).size
        query_db(f"INSERT INTO control (name, datetime, type, size) VALUES ('{file_name}', '{dt}', '{type_control}', '{sz}')")
        
        path_file = path + f'/{file_name}'
    
    else: #case the file has been unzipped
            
        path_file = False
        
    
    return path_file

# Checking Data

## Getting info

In [17]:
df = read_db("SELECT * FROM control WHERE type = 'extraction'")
down_df = check_control_db(file_type = 'download')

In [18]:
df['data'] = df['name'].str[:8].apply(lambda x: datetime.strptime(x, '%Y%m%d').strftime('%Y-%m-%d'))
df['data'] = pd.to_datetime(df['data'])
df['size'] = df['size'].astype(float)
df['count_total'] = df['count_total'].astype(float)
df['count_filtered'] = df['count_filtered'].astype(float)

In [19]:
donwloaded = df.groupby(pd.Grouper(key='data', freq='M'))['size'].sum()
print(f"Total of days: {len(down_df)} \n")

print(f"Total Downloaded: {humanize(donwloaded.sum())}")
print(f"An average of {humanize(donwloaded.sum()/len(down_df))} per day downloaded \n")

print("Total of Tweets: {:,}".format(df['count_total'].sum()))
print(f"An average of {df['count_total'].sum()/len(down_df):,.0f} tweets downloaded per day \n")

print("Tweets filtered: {:,}".format(df['count_filtered'].sum()))
print(f"An average of {df['count_filtered'].sum()/len(down_df):,.0f} tweets filtered per day \n")


#deleting df
del down_df, donwloaded, df

Total of days: 365 

Total Downloaded: 917.96 GiB
An average of 2.51 GiB per day downloaded 

Total of Tweets: 1,412,214,396.0
An average of 3,869,081 tweets downloaded per day 

Tweets filtered: 763,267.0
An average of 2,091 tweets filtered per day 



## Checking Duplicates

In [20]:
names_control = read_db(f"SELECT name FROM control")

In [21]:
duplicated = names_control.duplicated(['name'])
df_duplicated = names_control.loc[duplicated, :]
if len(df_duplicated)> 0 :
    print(f'Find {len(df_duplicated)} duplicated')
    print(df_duplicados)
else:
    print("Don't find duplicated") 
    
    
#deleting df
del duplicated, df_duplicated, names_control

Don't find duplicated


# Importing to Hadoop 

In [99]:
def mysql_to_hdfs(path, mysql_table, hdfs_folder = '/CA4' ,split_by_month=False, date_col='created_at'):

    path = check_folder(path)

    #Loading data from MySQL database
    df = read_db(f"SELECT * FROM {mysql_table}")
    
    if split_by_month == True:
        
        #Adding month
        df[date_col] = pd.to_datetime(df[date_col]).dt.tz_localize('UTC')
        df['month'] = df[date_col].dt.month

        # Splitting DataFrame by month and saving in parquet files 
        for month in range(1, 13):
            month_df = df[df['month'] == month]
            month_df.to_parquet(f"{path}/{mysql_table}_{month}.parquet")
            print(f"{mysql_table}_{month}.parquet >>> SAVED AT >>> {path}")
            
    else:
        df.to_parquet(f"{path}/{mysql_table}.parquet")
    
    # Create a client
    client = InsecureClient('http://localhost:9870', user='hduser')

    # Copy files to hdfs
    client.upload(hdfs_folder, path)
    print(f"{path} >>> UPLOADED AT >>> {hdfs_folder}")
    

## Control Table

In [88]:
mysql_to_hdfs(path = 'data/control', 
              mysql_table = 'control',
              hdfs_folder = '/CA4',
              split_by_month = False)

/home/hduser/Desktop/CA 2.2/Data Collect/data/control created.
/home/hduser/Desktop/CA 2.2/Data Collect/data/control >>> UPLOADED AT >>> /CA4


## Tweets Table

In [101]:
mysql_to_hdfs(path = 'data/tweets', 
              mysql_table = 'tweets',
              hdfs_folder = '/CA4',
              split_by_month = True,
              date_col='created_at')

tweets_1.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_2.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_3.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_4.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_5.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_6.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_7.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_8.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_9.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_10.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_11.parquet >>> SAVED AT >>> /home/hduser/Desktop/CA 2.2/Data Collect/data/tweets
tweets_12.parquet >>> SAVED AT >>> /home/